In [ ]:
# Cell 1: Imports (ML + AI)
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 

# --- Scikit-Learn (O que já tinhas) ---
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
# --- CORREÇÃO AQUI ---
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, RocCurveDisplay, f1_score

# --- NOVAS: TensorFlow / Keras (IA) ---
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# --- Opções de Display ---
pd.set_option('display.max_columns', 50)
sns.set_style('whitegrid')

print("All libraries imported successfully (including TensorFlow/Keras and f1_score)!")

In [ ]:
# Cell 2: Load CSV and get a quick overview
df = pd.read_csv('survey.csv')

# Show the dimensions of the dataset
print("Dataset shape:", df.shape)

# Display the first 6 rows
display(df.head(6))

# Display summary info about the dataset (column types, non-null counts, etc.)
display(df.info())

In [ ]:
# Cell 3: Exploratory Data Analysis (EDA)

# List all columns
print("Columns in the dataset:", list(df.columns))

# Show counts of each value in the target column ('treatment')
print("\nTarget value counts (treatment):")
print(df['treatment'].value_counts(dropna=False))

# Calculate percentage of missing values per column
missing_pct = df.isna().mean().sort_values(ascending=False) * 100
print("\nColumns with missing values (%):")
display(missing_pct[missing_pct > 0].round(2))

# Visualize the distribution of the target variable
plt.figure(figsize=(8,4))
sns.countplot(data=df, x='treatment', order=df['treatment'].value_counts().index)
plt.title('Distribution of the target: treatment')
plt.show()

# Visualize the distribution of age
plt.figure(figsize=(8,4))
sns.histplot(df['Age'].dropna(), bins=30)
plt.title('Age distribution')
plt.show()

In [ ]:
# Cell 4: Basic cleaning: Age, Gender normalization, and mapping Yes/No to 1/0
df2 = df.copy()  # work on a copy to preserve original data

# Age cleaning: convert to numeric and filter out unrealistic values

df2['Age'] = pd.to_numeric(df2['Age'], errors='coerce')  # convert to numeric, invalids become NaN
print("Before filtering: number of null ages =", df2['Age'].isna().sum())

# Optional: remove ages outside a realistic range (14-100)
df2.loc[(df2['Age'] < 14) | (df2['Age'] > 100), 'Age'] = np.nan
print("After filtering: number of null ages =", df2['Age'].isna().sum())


# Gender normalization: map variations to Male / Female / Other

def clean_gender(x):
    if pd.isna(x):
        return 'Other'
    s = str(x).strip().lower()
    # common male variants
    if s in ['male', 'm', 'man', 'male-ish', 'maile', 'mal', 'cis male', 'male (cis)']:
        return 'Male'
    # common female variants
    if s in ['female', 'f', 'woman', 'female (cis)', 'cis female']:
        return 'Female'
    # anything else (including 'trans' or 'non-binary') as Other
    return 'Other'

df2['Gender_clean'] = df2['Gender'].apply(clean_gender)


# Map binary Yes/No columns to 1/0

binary_cols = ['self_employed', 'family_history', 'treatment', 'remote_work', 'tech_company']

# --- NOTA: Esta lógica é frágil e foi a causa dos nossos erros. ---
# --- Vamos corrigi-la na Célula 7, como fizemos no outro notebook ---
with warnings.catch_warnings(): # Silenciar os FutureWarnings que isto vai dar
    warnings.simplefilter(action='ignore', category=FutureWarning)
    for col in binary_cols:
        if col in df2.columns:
            df2[col] = df2[col].map({'Yes': 1, 'No': 0})  # convert Yes/No to 1/0
            # keep existing numeric values as-is
            df2[col] = df2[col].fillna(df2[col]).infer_objects(copy=False)

print("Basic cleaning done. Sample data:")
display(df2[['Age','Gender','Gender_clean','self_employed','family_history','treatment']].head())

# (Gráficos omitidos para ser breve)

In [ ]:
# Cell 5: Initial features and simple new features
df3 = df2.copy()  # work on a new copy to preserve previous cleaning

# Example of a binary feature: long_hours (e.g., work hours > 50)
# Note: if there is no 'hours' column in the dataset, we skip this part.
# Here, we assume there is no 'hours' column, so we do not create it.

# List of candidate features to consider for analysis/modeling
candidate_features = [
    'Age', 
    'Gender_clean',
    'self_employed',
    'family_history',
    'work_interfere',   # categorical: 'Never','Rarely','Sometimes','Often'
    'no_employees',     # categorical: company size
    'remote_work',
    'tech_company',
    'benefits',
    'care_options',
    'wellness_program',
    'seek_help',
    'anonymity'
]

# Keep only the features that exist in the current dataframe
candidate_features = [c for c in candidate_features if c in df3.columns]
print("Candidate features used:", candidate_features)

# Quick peek at unique values for categorical features (or those with <20 unique values)
# --- ESTA É A PARTE QUE TINHA SIDO OMITIDA ---
for col in candidate_features:
    if df3[col].dtype == 'object' or df3[col].nunique() < 20:
        print(f"\n--- {col} unique values ---")
        print(df3[col].fillna('NA').value_counts().head(10))

In [ ]:
# Cell 6: Preprocessing Pipeline (A Versão "Corrigida" do TypeError)
# Esta é a célula que falhou no teu notebook 1 original.
# Vamos usar a versão CORRIGIDA que descobrimos, que inclui o FunctionTransformer.

print("A definir o preprocessor (com o fix .astype(str))...")

# Separate numerical and categorical features
num_features = [c for c in candidate_features if df3[c].dtype in ['int64','float64'] and c != 'treatment']
cat_features = [c for c in candidate_features if c not in num_features]

print("Numerical features:", num_features)
print("Categorical features:", cat_features)

# Numerical pipeline: impute missing values with median, then scale
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical pipeline: (COM A CORREÇÃO DO TypeError)
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('to_string', FunctionTransformer(lambda x: x.astype(str))), # <-- O FIX MÁGICO
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine pipelines using ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
], remainder='drop', sparse_threshold=0)

print("Preprocessor (corrigido) criado.")

In [ ]:
# Cell 7: Prepare X,y and separate training/test (A Versão "Corrigida" do ValueError)
# Esta é a outra célula que falhou. Vamos usar a versão ROBUSTA.

print("A preparar X e y (com o fix pd.to_numeric)...")
X = df3[candidate_features].copy()

# --- CORREÇÃO (Lógica 'pd.to_numeric' Robusta) ---
# A lógica .map() antiga falhava e criava 100% de NaNs.
y_temp = df3['treatment'].replace({'Yes': 1, 'No': 0})
y = pd.to_numeric(y_temp, errors='coerce')
# --- FIM DA CORREÇÃO ---

# Remove rows with missing target values
mask = y.notna()
X = X[mask]
y = y[mask].astype(int) # .astype(int) agora funciona

print(f"Shape after cleaning NaNs from target: X={X.shape}, y={y.shape}")

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Train/test shapes:", X_train.shape, X_test.shape)
print("Train target dist:", np.bincount(y_train)/len(y_train))

In [ ]:
# Cell 8: Preprocess Data and Define NN "Raw" Functions

# 1. Aplicar o preprocessor
preprocessor.fit(X_train)
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# 2. Reformular os dados para NumPy (como na tua aula)
# O Keras quer (n_amostras, n_features). O NumPy "raw" quer (n_features, n_amostras).
# Vamos "transpor" (Transpose) os dados.
X_train_raw = X_train_processed.T
X_test_raw = X_test_processed.T

# O 'y' tem de ser um "array" de 1 linha
y_train_raw = y_train.values.reshape(1, y_train.shape[0])
y_test_raw = y_test.values.reshape(1, y_test.shape[0])

print(f"Formato 'Raw' de X_train (features, amostras): {X_train_raw.shape}")
print(f"Formato 'Raw' de y_train (1, amostras): {y_train_raw.shape}")

# 3. Funções "Helper" (baseadas na Aula L2)

def sigmoid(z):
    """Calcula a função sigmóide (logística) [cite: 325]"""
    return 1 / (1 + np.exp(-z))

def sigmoid_gradient(z):
    """Calcula a derivada da sigmóide (para a retropropagação) [cite: 496]"""
    s = sigmoid(z)
    return s * (1 - s)

In [ ]:
# Cell 9: Initialize Parameters
def initialize_parameters(n_x, n_h, n_y):
    """
    Inicializa os pesos (Thetas) com valores aleatórios pequenos.
    n_x: neurónios na camada de input (as nossas features)
    n_h: neurónios na camada oculta
    n_y: neurónios na camada de output (para nós, é 1)
    """
    np.random.seed(42) # Para reprodutibilidade
    
    # W1 é o nosso Theta_1 (camada 1 -> 2)
    # W2 é o nosso Theta_2 (camada 2 -> 3)
    # 
    W1 = np.random.randn(n_h, n_x) * 0.01 # Quebra a simetria [cite: 534]
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    
    return parameters

In [ ]:
# Cell 10: Forward Propagation
def forward_propagation(X, parameters):
    """
    Executa o "forward pass" através da rede. [cite: 451]
    """
    # Retira os pesos da "gaveta"
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # Camada 1 -> Camada 2 (Oculta)
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1) # Usar 'tanh' como ativação (uma opção da Aula L2) [cite: 326]
    
    # Camada 2 -> Camada 3 (Output)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2) # Usar 'sigmoid' na saída para classificação binária [cite: 325]
    
    # Guarda os valores intermédios (cache) para usar na retropropagação
    cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}
    
    return A2, cache

In [ ]:
# Cell 11: Compute Cost Function (with Regularization)
def compute_cost(A2, Y, parameters, lambda_reg):
    """
    Calcula o custo (Binary Cross-Entropy) + Regularização L2. [cite: 382, 401]
    A2: As nossas previsões (saída da sigmoid)
    Y: As respostas corretas (0 ou 1)
    lambda_reg: O "lambda" da regularização
    """
    m = Y.shape[1] # Número de exemplos
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    
    # Custo de Cross-Entropy [cite: 380]
    logprobs = np.multiply(np.log(A2), Y) + np.multiply(np.log(1 - A2), (1 - Y))
    cross_entropy_cost = - (1 / m) * np.sum(logprobs)
    
    # Custo da Regularização L2 [cite: 401]
    L2_cost = (lambda_reg / (2 * m)) * (np.sum(np.square(W1)) + np.sum(np.square(W2)))
    
    total_cost = cross_entropy_cost + L2_cost
    
    return np.squeeze(total_cost) # Garante que é só um número

In [ ]:
# Cell 12: Backward Propagation
def backward_propagation(parameters, cache, X, Y, lambda_reg):
    """
    Implementa o algoritmo de Error Backpropagation. [cite: 474]
    """
    m = X.shape[1]
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    
    A1 = cache['A1']
    A2 = cache['A2']
    Z1 = cache['Z1']
    
    # 1. Erro na Camada de Saída (delta 3) [cite: 479]
    dZ2 = A2 - Y
    
    # 2. Gradientes da Camada 2-3
    dW2 = (1 / m) * np.dot(dZ2, A1.T) + (lambda_reg / m) * W2 # Adiciona grad. da regularização
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    
    # 3. Retropropagar o Erro (delta 2) [cite: 482]
    # (Usamos a derivada da 'tanh', que é 1 - A1^2)
    dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
    
    # 4. Gradientes da Camada 1-2
    dW1 = (1 / m) * np.dot(dZ1, X.T) + (lambda_reg / m) * W1 # Adiciona grad. da regularização
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    
    return grads

In [ ]:
# Cell 13: Update Parameters (Gradient Descent)
def update_parameters(parameters, grads, learning_rate):
    """
    Atualiza os pesos usando os gradientes. [cite: 486]
    """
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    # A regra de atualização
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    
    return parameters

In [ ]:
# Cell 14: The Full Neural Network Model
def nn_model(X, Y, n_h, num_epochs, lambda_reg, learning_rate, print_cost=False):
    """
    Constrói e treina o modelo de NN "raw".
    """
    n_x = X.shape[0] # n_features
    n_y = Y.shape[0] # n_outputs (1)
    costs = []
    
    # 1. Inicializar parâmetros
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    # 2. Loop de Treino (Epochs) 
    for i in range(0, num_epochs):
        
        # 3. Forward propagation [cite: 437]
        A2, cache = forward_propagation(X, parameters)
        
        # 4. Calcular Custo [cite: 378]
        cost = compute_cost(A2, Y, parameters, lambda_reg)
        
        # 5. Backward propagation [cite: 454]
        grads = backward_propagation(parameters, cache, X, Y, lambda_reg)
        
        # 6. Atualizar parâmetros (Gradient Descent) [cite: 507]
        parameters = update_parameters(parameters, grads, learning_rate)
        
        if print_cost and i % 100 == 0:
            print(f"Custo após epoch {i}: {cost}")
            costs.append(cost)
            
    return parameters, costs

# Função para fazer previsões
def predict(X, parameters):
    A2, cache = forward_propagation(X, parameters)
    predictions = (A2 > 0.5).astype(int) # Arredonda para 0 ou 1
    return predictions

In [ ]:
# Cell 15: Train the "Raw" MLP

# --- Hiperparâmetros ---
n_h = 8                # 8 neurónios na camada oculta
num_epochs = 2000      # Número de "rondas" de treino 
lambda_reg = 0.1       # Força da regularização (o nosso 'lambda') [cite: 394]
learning_rate = 0.01   # Taxa de aprendizagem (o nosso 'alpha') [cite: 508]

print("A iniciar treino do modelo 'raw'...")
start_time = time.time()

parameters, costs = nn_model(
    X_train_raw, y_train_raw, 
    n_h=n_h, 
    num_epochs=num_epochs, 
    lambda_reg=lambda_reg, 
    learning_rate=learning_rate, 
    print_cost=True
)

print(f"Treino 'raw' demorou {time.time() - start_time:.2f} segundos.")

# Plotar a curva de custo (para ver se aprendeu)
plt.plot(np.squeeze(costs))
plt.ylabel('Custo (Loss)')
plt.xlabel('Epochs (x100)')
plt.title(f"Curva de Aprendizagem (Learning Rate = {learning_rate})")
plt.show()

In [ ]:
# Cell 16: Evaluate the "Raw" MLP

# 1. Fazer previsões no set de TESTE
y_pred_raw_test = predict(X_test_raw, parameters)

# 2. Fazer previsões no set de TREINO (para ver se há overfitting)
y_pred_raw_train = predict(X_train_raw, parameters)

# 3. O y_pred_raw é (1, n_amostras). O y_test é (n_amostras,).
# Temos de "achatar" (flatten) as previsões para os comparar.
y_pred_flat_test = y_pred_raw_test.flatten()
y_pred_flat_train = y_pred_raw_train.flatten()


# [cite_start]--- Diagnosticar Bias vs. Variance (Aula L3) [cite: 848] ---
print("\n--- Diagnóstico de Performance (Aula L3) ---")
print("Relatório de Classificação (TREINO):")
print(classification_report(y_train, y_pred_flat_train, digits=4))
print("---")
print("Relatório de Classificação (TESTE):")

# --- A CORREÇÃO ESTÁ AQUI ---
# Trocámos y_pred_flat_TRAIN por y_pred_flat_TEST
print(classification_report(y_test, y_pred_flat_test, digits=4)) 
# --- FIM DA CORREÇÃO ---

# [cite_start]Se o F1 de Treino for 0.99 e o de Teste 0.80 -> High Variance (Overfitting) [cite: 852]
# [cite_start]Se o F1 de Treino for 0.70 e o de Teste 0.69 -> High Bias (Underfitting) [cite: 850]

# [cite_start]--- Matriz de Confusão (Aula L3) [cite: 554] ---
print("\nMatriz de Confusão (TESTE):")
cm = confusion_matrix(y_test, y_pred_flat_test) # Usa a variável correta aqui também
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No', 'Yes'])
disp.plot(cmap=plt.cm.Blues, colorbar=False)
disp.ax_.grid(False)
plt.show()

In [ ]:
# Cell 17: O Modelo "Refinado" (com Early Stopping)

# Esta é uma CÓPIA da 'nn_model' (Célula 14),
# mas modificada para incluir Early Stopping.
def nn_model_refined(X_train, Y_train, X_val, Y_val, n_h, num_epochs, lambda_reg, learning_rate, patience=20):
    """
    Constrói e treina o modelo "raw" com Early Stopping.
    X_val, Y_val: Dados de validação para verificar o overfitting
    patience: Quantas epochs esperar sem melhoria
    """
    
    n_x = X_train.shape[0]
    n_y = Y_train.shape[0]
    costs_train = []
    costs_val = [] # Guardar o custo de validação
    
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    best_cost = float('inf')
    patience_counter = 0
    best_params = {}
    
    for i in range(0, num_epochs):
        
        # --- Treino ---
        A2_train, cache_train = forward_propagation(X_train, parameters)
        cost_train = compute_cost(A2_train, Y_train, parameters, lambda_reg)
        grads = backward_propagation(parameters, cache_train, X_train, Y_train, lambda_reg)
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # --- Validação ---
        A2_val, _ = forward_propagation(X_val, parameters)
        cost_val = compute_cost(A2_val, Y_val, parameters, lambda_reg)
        
        costs_train.append(cost_train)
        costs_val.append(cost_val)

        # --- Lógica de Early Stopping (Sugestão 5) ---
        if cost_val < best_cost:
            best_cost = cost_val
            best_params = parameters.copy() # Guardar os melhores pesos
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f"--- Early stopping na epoch {i} ---")
            break
            
    return best_params, costs_train, costs_val

print("Função 'nn_model_refined' (com Early Stopping) definida.")

In [ ]:
# Cell 18: Grid Search Manual (Sugestão 4)

print("A iniciar Grid Search manual para o modelo refinado...")

# 1. Definir a Grelha
# (É uma grelha mais pequena para ser rápida. Podes aumentar mais tarde)
param_grid = {
    'n_h': [8, 16], # Neurónios na camada oculta
    'lambda_reg': [0.1, 0.5, 1.0], # Força da regularização
    'learning_rate': [0.01] # Taxa de aprendizagem
}

best_score_f1 = -1
best_hyperparams = {}
best_model_params = {}

start_time = time.time()

# 2. Loop de Grid Search
for n_h in param_grid['n_h']:
    for lr in param_grid['learning_rate']:
        for lam in param_grid['lambda_reg']:
            
            print(f"\nA testar: n_h={n_h}, lr={lr}, lambda={lam}")
            
            # Treinar o modelo refinado
            # (Usamos X_test_raw/y_test_raw como set de validação)
            params, _, _ = nn_model_refined(
                X_train_raw, y_train_raw,
                X_test_raw, y_test_raw, # Usa o set de teste como validação
                n_h=n_h, 
                num_epochs=2000, # Epochs altas (o Early Stopping trata de parar)
                lambda_reg=lam, 
                learning_rate=lr,
                patience=50 # Paciência de 50 epochs
            )
            
            # 3. Avaliar
            # (Temos de "achatar" as previsões, como na Célula 16)
            y_pred_val = predict(X_test_raw, params).flatten()
            
            # Usar o f1-score (como no Random Forest)
            score = f1_score(y_test, y_pred_val) 
            
            if score > best_score_f1:
                best_score_f1 = score
                best_hyperparams = {'n_h': n_h, 'lr': lr, 'lambda': lam}
                best_model_params = params # Salva os pesos (Thetas) do melhor modelo

print(f"\nGrid Search demorou {time.time() - start_time:.2f} segundos.")
print(f"\n--- MELHOR MODELO (REFINADO) ---")
print(f"Melhor F1-score (na validação): {best_score_f1:.4f}")
print(f"Melhores Hiperparâmetros: {best_hyperparams}")

In [ ]:
# Cell 19: Avaliação Final do Modelo Refinado (Sugestões 6 e 7)

# 1. Fazer previsões no set de TESTE com o MELHOR modelo
y_pred_refined_test = predict(X_test_raw, best_model_params).flatten()
y_pred_refined_train = predict(X_train_raw, best_model_params).flatten()

print("\n--- Diagnóstico de Performance (MODELO REFINADO) ---")
print("Relatório de Classificação (TREINO - Refinado):")
print(classification_report(y_train, y_pred_refined_train, digits=4))
print("---")
print("Relatório de Classificação (TESTE - Refinado):")
print(classification_report(y_test, y_pred_refined_test, digits=4))
print("---")

# 2. Matriz de Confusão
print("\nMatriz de Confusão (TESTE - Refinado):")
cm = confusion_matrix(y_test, y_pred_refined_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No', 'Yes'])
disp.plot(cmap=plt.cm.Blues, colorbar=False)
disp.ax_.grid(False)
plt.show()

# 3. Visualização (Sugestão 6): Distribuição de Probabilidades
print("\nDistribuição das Probabilidades Previstas (TESTE - Refinado):")
# (Temos de correr o forward_propagation para ir buscar as probabilidades 'A2')
probs_refined, _ = forward_propagation(X_test_raw, best_model_params)
probs_flat = probs_refined.flatten()

plt.figure(figsize=(10, 6))
plt.hist(probs_flat[y_test == 0], bins=30, alpha=0.5, label='Classe 0 (No)')
plt.hist(probs_flat[y_test == 1], bins=30, alpha=0.5, label='Classe 1 (Yes)')
plt.xlabel('Probabilidade Prevista de ser "Yes"')
plt.ylabel('Frequência')
plt.title('Distribuição de Probabilidades (Modelo Refinado)')
plt.legend()
plt.show()

# 4. Análise de Erros (Sugestão 7)
print("\n--- Análise de Erros (TESTE - Refinado) ---")
misclassified_idx = np.where(y_test != y_pred_refined_test)[0]
print(f"Total de erros: {len(misclassified_idx)} de {len(y_test)}")
print("A mostrar os 5 primeiros erros:")

for idx in misclassified_idx[:5]:
    original_idx = y_test.index[idx] # Pega no índice original do DataFrame
    true_label = y_test.iloc[idx]
    pred_label = y_pred_refined_test[idx]
    
    print(f"\n* Índice (original): {original_idx}")
    print(f"  Verdade: {true_label}, Previsão: {pred_label}")
    # O 'X' (DataFrame original) tem o índice correto
    print(f"  Features: {X.loc[original_idx].to_dict()}")

In [ ]:
# Cell 21: Análise de Importância da Feature (Modelo REFINADO)

print("--- Análise de Importância (Pesos da Camada 1 do Modelo REFINADO) ---")

# --- A CORREÇÃO ESTÁ AQUI ---
# 1. Ir buscar os nomes das features (em vez de depender de outra célula)
# (Estas variáveis 'preprocessor', 'num_features', 'cat_features' vêm da Célula 6)
print("A ir buscar os nomes das features do preprocessor...")
onehot_features = preprocessor.transformers_[1][1].named_steps['onehot'].get_feature_names_out(cat_features)
all_features = num_features + list(onehot_features)
print(f"Encontradas {len(all_features)} features no total.")
# --- FIM DA CORREÇÃO ---

# 2. Ir buscar a matriz de pesos W1 (Theta 1) do *melhor* modelo
# (best_model_params foi criado na Célula 18)
W1_refined = best_model_params['W1'] 

# 3. Calcular a importância
feature_importance_refined = np.mean(np.abs(W1_refined), axis=0)

# 4. Criar um DataFrame bonito para ver
df_importance_refined = pd.DataFrame({
    'Feature': all_features,
    'Importancia (Media Abs W1)': feature_importance_refined
})

# 5. Mostrar o Top 10
print("\nTop 10 Features (baseado na magnitude média dos pesos da 1ª camada):")
display(df_importance_refined.sort_values(by='Importancia (Media Abs W1)', ascending=False).head(10))